Data Processing

Filter HTML files

In [6]:
import sqlite3
import pandas as pd

In [7]:
database = 'html.sqlite'
sql = """
SELECT filename, title, text
FROM html_attrs
"""

con = sqlite3.connect(database)
html_df = pd.read_sql_query(sql, con)
con.close()

html_df.dtypes

filename    object
title       object
text        object
dtype: object

Anfrage:
    Schlüsselwörter

In [9]:
word = "Schiedermeier"

[text for text in html_df["text"] if word in text][:5]

['       Allgemeine Studienberatung  Allgemeine Beratung erhalten Sie von der zentralen  Studienberatung. Die persönlichen Anforderungen, die für ein erfolgreiches Studium wichtig sind, haben wir zusammengestellt.        Studienfachberatung Informatik, Medieninformatik und Wirtschaftsinformatik  Wir wollen, dass Sie erfolgreich studieren! Wir beraten Sie vor Beginn und während des Studiums. Wir informieren Sie in regelmäßigen Veranstaltungen. Wir vermitteln Ansprechpartner wie Professoren, Kommissionen, Externe. Sie erreichen uns zu den ausgehängten Sprechzeiten bzw. über Telefon oder per E-Mail.            Bachelor - Studiengang Informatik - Studiengang Medieninformatik       Bachelor - Studiengang Informatik - Studiengang Medieninformatik         Name  Kontakt            Christian  Schiedermeier     Prof. Dr.-Ing.      telefon  +49 (0)911 5880 - 1179     email  christian.schiedermeier at th-nuernberg Punkt de     fax  +49 (0)911 5880 - 5666               Bachelor - Studiengang Wirtsc

Word embeddings für jedes Dokument erstellen.
BERT Embeddings
BERT (Google) trainiert auf 10.000+ Büchern
es gibt Modelle "base" und "large"
uncased heißt ohne klein - Großschreibung

Wir brauchen zur vorbereitung die zusätzlichen Token
[SEP] um das Ende eines Satzes zu markieren
[CLS] am Anfang des Texten
[PAD] zum auffüllen der Token 
Außerdem
TokenIDs
MaskIDs - zum filtern der [PAD]
Segment IDs um verschiedene Sätze zu unterscheiden
Posititional Embeddings


In [10]:
from transformers import BertModel, BertTokenizer
import torch

/Users/br/Projects/IT-Ptojekt-Chatbot/daibl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model = BertModel.from_pretrained('bert-base-uncased')

testSentence = "In der Bibliothek gibt es 40 Bücher zu Thema Animes"

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer.tokenize(testSentence)

In [12]:
html_df["tokens"] = [tokenizer.tokenize(text) for text in html_df["text"]]

In [15]:
print(html_df["text"][2])
print(html_df["tokens"][2])

       Eine Pilotstudie. Oder: Wer kümmert sich um das Ehrenamt?  In der Praxis engagementfördernder Strukturen leisten „Verantwortliche für Engagement“ in Kommunen, Vereinen und Verbänden einen wichtigen Beitrag zum Gelingen des Engagements. Seit längerem ist dies u.a. ein Feld der Professionalisierung für Absolvent:innen des Studiengangs Soziale Arbeit. In der bisherigen Forschung wurde deren berufliches Setting jedoch kaum beleuchtet. Das Pilotprojekt wird hier erste valide Erkenntnisse bringen.    Laufzeit: 1. April 2022 bis 31. März 2023            Beteiligte         Projektleitung: Prof. Dr. Doris Rosenkranz (Fakultät SW) Projektmitarbeiter/in: Enya Buchner, M.A.           Zielsetzung         Verantwortliche für Engagement gibt es bundesweit in jedem Feld des Engagements. Doch bereits im „Wording“ für diese Funktionen zeigt sich eine große Heterogenität. Die These: Nach Organisation, Rahmenbedingungen oder Engagementfeld unterscheiden sich Bezeichnungen, berufliche Settings, Zuga

In [ ]:

print(tokens)

['[CLS]', 'in', 'der', 'bi', '##bl', '##iot', '##he', '##k', 'gi', '##bt', 'es', '40', 'bu', '##cher', 'zu', 'them', '##a', 'anime', '##s', '[SEP]']


In [ ]:
#tokens = ['[CLS]'] + tokens + ['[SEP]']

In [ ]:
attention_mask = [1 if token != "[PAD]" else 0  for token in tokens]

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[101, 1999, 4315, 12170, 16558, 25185, 5369, 2243, 21025, 19279, 9686, 2871, 20934, 7474, 16950, 2068, 2050, 8750, 2015, 102]


In [ ]:
token_ids = torch.tensor(token_ids).unsqueeze(0)

attention_mask = torch.tensor(attention_mask).unsqueeze(0)

In [ ]:
output = model(token_ids, attention_mask=attention_mask)

In [ ]:
print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4551,  0.4743, -0.1221,  ..., -0.3652,  0.5639,  0.4320],
         [-0.6011,  0.2618, -0.4274,  ..., -0.2823,  0.0178,  0.0410],
         [-0.6369,  0.0607, -0.3814,  ..., -0.3802,  0.0530,  0.0642],
         ...,
         [ 0.1221,  0.3641, -0.0675,  ..., -0.0359,  0.7755,  0.1731],
         [-0.5679, -0.1522, -0.1363,  ..., -0.3275,  0.3004, -0.2525],
         [ 0.7437,  0.2021, -0.2943,  ...,  0.0588, -0.6261,  0.0194]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.1740e-01, -2.8785e-01, -2.5498e-01,  7.0340e-01, -4.3939e-03,
         -1.1000e-01,  6.2925e-01,  2.8865e-01,  4.1364e-02, -9.9987e-01,
          6.1949e-02,  5.4157e-01,  9.8279e-01,  1.2675e-02,  7.8739e-01,
         -5.1049e-01,  1.7213e-01, -4.6361e-01,  3.4037e-01,  8.5037e-02,
          4.9879e-01,  9.9920e-01,  4.6352e-01,  2.3775e-01,  3.4453e-01,
          6.2686e-01, -6.3081e-01,  9.2418e-01,  9.4652e-01,  6.257

In [ ]:
output[0].shape

torch.Size([1, 20, 768])